In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 
import os
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.model_selection import train_test_split
os.environ['TF_CPP_MIN_LOG_LEVEL']='2'
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory
import subprocess
from subprocess import check_output
try:
    print(check_output(["ls", "../v2"]).decode("utf8"))
except subprocess.CalledProcessError as e:
    raise RuntimeError("command '{}' return with error (code {}): {}".format(e.cmd, e.returncode, e.output))
# Any results you write to the current directory are saved as output.
import tensorflow as tf

frame
video



In [2]:
# now, let's read the frame-level data
# due to execution time, we're only going to read the first video
video_files = os.listdir("../v2/video/")
frame_files = os.listdir("../v2/frame/")

# Global variables
num_labels = 4716

In [3]:
def createTargetVec(labels):
    out = np.zeros((1, num_labels))
    for label in labels:
        out[0,label] = 1
    return out

In [4]:
def get_dataset(frames, samples=5, k=0):
    rgb_input = np.empty((samples, 100, 1024))
    audio_input = np.empty((samples, 100, 128))
    label_output = np.empty((samples, num_labels))
    sess = tf.InteractiveSession()
    for example in tf.python_io.tf_record_iterator(frames):        
        tf_seq_example = tf.train.SequenceExample.FromString(example)
        labels = tf_seq_example.context.feature['labels'].int64_list.value
        rgb_frame = np.zeros((100, 1024))
        audio_frame = np.zeros((100, 128))
        for i in range(100):
            rgb_frame[i] = tf.cast(tf.decode_raw(
                   tf_seq_example.feature_lists.feature_list['rgb'].feature[i].bytes_list.value[0],tf.uint8)
                          ,tf.float32).eval()
            audio_frame[i] = tf.cast(tf.decode_raw(
                    tf_seq_example.feature_lists.feature_list['audio'].feature[i].bytes_list.value[0],tf.uint8)
                           ,tf.float32).eval()
        rgb_input[k] = rgb_frame
        audio_input[k] = audio_frame
        label_output[k] = createTargetVec(labels[:])
        k += 1
        progress = (k / samples) * 100
        if int(progress) % 10 == 0:
            print("Progress", progress, "%")
        if k >= samples:
            break
    sess.close()
    return audio_input, rgb_input, label_output

In [5]:
# video_lvl_record = "../v2/video/train-1.tfrecord"
frame_lvl_record = "../v2/frame/train4f.tfrecord"

audio_input, rgb_input, label_output = get_dataset(frame_lvl_record)

X_train_rgb, X_test_rgb, _, _ = train_test_split(rgb_input, label_output, test_size=0.2, random_state=42)
X_train_audio, X_test_audio, y_train, y_test = train_test_split(audio_input, label_output, test_size=0.2, random_state=42)

Progress 20.0 %
Progress 40.0 %
Progress 60.0 %
Progress 80.0 %
Progress 100.0 %


In [6]:
print(len(X_train_rgb), 'train rgb')
print(len(X_test_rgb), 'test rgb')
print("")
print(len(X_train_audio), 'train sequences')
print(len(X_test_audio), 'test sequences')
print("")
print('X_train_rgb shape:', X_train_rgb.shape)
print('X_test_rgb shape:', X_test_rgb.shape)
print("")
print('X_train_audio shape:', X_train_audio.shape)
print('X_test_audio shape:', X_test_audio.shape)
print("")
print('y_train shape:', y_train.shape)
print('y_test shape:', y_test.shape)

4 train rgb
1 test rgb

4 train sequences
1 test sequences

X_train_rgb shape: (4, 100, 1024)
X_test_rgb shape: (1, 100, 1024)

X_train_audio shape: (4, 100, 128)
X_test_audio shape: (1, 100, 128)

y_train shape: (4, 4716)
y_test shape: (1, 4716)


## Lets take a look at the rgb data
****

In [7]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

In [8]:
rgb_input

array([[[100.,  54.,  84., ...,  46.,  10., 132.],
        [105.,   0.,  91., ..., 153.,  27., 103.],
        [ 93.,  24.,  95., ...,  84., 113., 130.],
        ...,
        [ 90.,  35., 122., ..., 190., 155., 147.],
        [ 86.,  26., 112., ..., 129., 100., 126.],
        [ 81.,  12., 120., ..., 222., 114., 140.]],

       [[ 65., 186., 255., ..., 151.,  58., 141.],
        [ 51., 198., 255., ..., 144.,  63., 167.],
        [ 89., 175., 255., ..., 255.,  80., 119.],
        ...,
        [148., 169., 255., ...,  13., 158., 101.],
        [159., 156., 203., ..., 103., 186.,  56.],
        [118., 176., 207., ..., 133., 156., 106.]],

       [[  0.,  85., 148., ..., 124., 135., 133.],
        [  0.,  62., 104., ..., 116., 111., 128.],
        [  0.,  75., 103., ...,  74.,  26., 176.],
        ...,
        [144., 172.,  86., ..., 244.,  72.,  95.],
        [210., 153., 100., ..., 255., 200., 189.],
        [200., 160.,  99., ..., 194., 220., 195.]],

       [[161.,  57.,  93., ...,  44.,

In [9]:
rgb_input.shape

(5, 100, 1024)

In [10]:
rgb_input[0]

array([[100.,  54.,  84., ...,  46.,  10., 132.],
       [105.,   0.,  91., ..., 153.,  27., 103.],
       [ 93.,  24.,  95., ...,  84., 113., 130.],
       ...,
       [ 90.,  35., 122., ..., 190., 155., 147.],
       [ 86.,  26., 112., ..., 129., 100., 126.],
       [ 81.,  12., 120., ..., 222., 114., 140.]])

In [11]:
rgb_input[0].shape

(100, 1024)

In [12]:
print('The first video has %d frames' %len(rgb_input[0]))

The first video has 100 frames


In [13]:
rgb_input[0][0]

array([100.,  54.,  84., ...,  46.,  10., 132.])

In [14]:
rgb_input[0][0].shape

(1024,)

In [15]:
import numpy as np
from matplotlib import pyplot as plt
from PIL import Image
import glob
import cv2

In [16]:
# x_data = np.array( [np.array(cv2.imread(rgb_input[0][0])) for i in range(len(rgb_input[0][0]))] )
# print (x_data.shape)

## Now anylyze the Audio data
****
** Since we're given sequential data, most likely I'll use an LSTM or GRU**

In [17]:
print(X_train_audio.shape[1:])

(100, 128)


In [18]:
from keras.layers import LSTM
from keras.layers import Dense
from keras.models import Sequential

audio_model = Sequential()
audio_model.add(LSTM(128, input_shape=(100,128), return_sequences=True))
audio_model.add(LSTM(128, return_sequences=True))
audio_model.add(LSTM(128))
audio_model.add(Dense(128, activation='relu'))
audio_model.add(Dense(128, activation='relu'))
audio_model.add(Dense(num_labels, activation='softmax'))
audio_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

Using TensorFlow backend.


In [19]:
print(audio_model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 100, 128)          131584    
_________________________________________________________________
lstm_2 (LSTM)                (None, 100, 128)          131584    
_________________________________________________________________
lstm_3 (LSTM)                (None, 128)               131584    
_________________________________________________________________
dense_1 (Dense)              (None, 128)               16512     
_________________________________________________________________
dense_2 (Dense)              (None, 128)               16512     
_________________________________________________________________
dense_3 (Dense)              (None, 4716)              608364    
Total params: 1,036,140
Trainable params: 1,036,140
Non-trainable params: 0
_________________________________________________________________


In [26]:
audio_model.fit(X_train_audio, y_train, validation_data=(X_test_audio, y_test), epochs=60, batch_size=64)
# evaluate the model
audio_scores = audio_model.evaluate(X_train_audio, y_train, verbose=0)
print("")
print("%s: %.2f%%" % (audio_model.metrics_names[1], audio_scores[1]*100))

Train on 4 samples, validate on 1 samples
Epoch 1/60
4/4 [==============================] - 1s 187ms/step - loss: 6.1311 - acc: 0.5000 - val_loss: 64.4724 - val_acc: 0.0000e+00
Epoch 2/60
4/4 [==============================] - 0s 84ms/step - loss: 6.1231 - acc: 0.5000 - val_loss: 64.4724 - val_acc: 0.0000e+00
Epoch 3/60
4/4 [==============================] - 0s 84ms/step - loss: 6.1144 - acc: 0.5000 - val_loss: 64.4724 - val_acc: 0.0000e+00
Epoch 4/60
4/4 [==============================] - 0s 86ms/step - loss: 6.1049 - acc: 0.5000 - val_loss: 64.4724 - val_acc: 0.0000e+00
Epoch 5/60
4/4 [==============================] - 0s 92ms/step - loss: 6.0946 - acc: 0.5000 - val_loss: 64.4724 - val_acc: 0.0000e+00
Epoch 6/60
4/4 [==============================] - 0s 105ms/step - loss: 6.0832 - acc: 0.5000 - val_loss: 64.4724 - val_acc: 0.0000e+00
Epoch 7/60
4/4 [==============================] - 0s 80ms/step - loss: 6.0700 - acc: 0.5000 - val_loss: 64.4724 - val_acc: 0.0000e+00
Epoch 8/60
4/4 [==

In [27]:
audio_model_json = audio_model.to_json()

In [23]:
with open("audio_model.json", "w") as json_file:
    json_file.write(audio_model_json)
# serialize weights to HDF5
audio_model.save_weights("audio_model.h5")
print("Saved model to disk")

Saved model to disk


In [31]:
# load json and create model
from keras.models import model_from_json
audio_json_file = open('audio_model.json', 'r')
audio_loaded_model_json = audio_json_file.read()
audio_json_file.close()
audio_loaded_model = model_from_json(audio_loaded_model_json)
# load weights into new model
audio_loaded_model.load_weights("audio_model.h5")
print("Loaded model from disk")

Loaded model from disk


In [32]:
# evaluate loaded model on test data
audio_loaded_model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
audio_score = audio_loaded_model.evaluate(X_train_audio, y_train, verbose=0)
print("%s: %.2f%%" % (audio_loaded_model.metrics_names[1], audio_score[1]*100))

acc: 99.94%


## Time for RGB Array
** **
** The dimensions for an rgb array is 1024 x 100. 100 seconds with each second having 1024 each. **

In [33]:
rgb_model = Sequential()
rgb_model.add(LSTM(1024, input_shape=(100,1024), return_sequences=True))
rgb_model.add(LSTM(1024, return_sequences=True))
rgb_model.add(LSTM(1024))
rgb_model.add(Dense(1024, activation='relu'))
rgb_model.add(Dense(1024, activation='relu'))
rgb_model.add(Dense(num_labels, activation='softmax'))
rgb_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [34]:
print(rgb_model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_4 (LSTM)                (None, 100, 1024)         8392704   
_________________________________________________________________
lstm_5 (LSTM)                (None, 100, 1024)         8392704   
_________________________________________________________________
lstm_6 (LSTM)                (None, 1024)              8392704   
_________________________________________________________________
dense_4 (Dense)              (None, 1024)              1049600   
_________________________________________________________________
dense_5 (Dense)              (None, 1024)              1049600   
_________________________________________________________________
dense_6 (Dense)              (None, 4716)              4833900   
Total params: 32,111,212
Trainable params: 32,111,212
Non-trainable params: 0
________________________________________________________________

In [35]:
rgb_model.fit(X_train_rgb, y_train, validation_data=(X_test_rgb, y_test), epochs=15, batch_size=64)
# evaluate the model
rgb_scores = rgb_model.evaluate(X_train_rgb, y_train, verbose=0)
print("")
print("%s: %.2f%%" % (rgb_model.metrics_names[1], rgb_scores[1]*100))

Train on 4 samples, validate on 1 samples
Epoch 1/15
4/4 [==============================] - 14s 4s/step - loss: 23.2864 - acc: 0.0000e+00 - val_loss: 33.4403 - val_acc: 0.0000e+00
Epoch 2/15
4/4 [==============================] - 10s 3s/step - loss: 22.2407 - acc: 0.5000 - val_loss: 32.3331 - val_acc: 0.0000e+00
Epoch 3/15
4/4 [==============================] - 11s 3s/step - loss: 15.1650 - acc: 0.5000 - val_loss: 46.0815 - val_acc: 0.0000e+00
Epoch 4/15
4/4 [==============================] - 11s 3s/step - loss: 11.3757 - acc: 0.5000 - val_loss: 45.5355 - val_acc: 0.0000e+00
Epoch 5/15
4/4 [==============================] - 10s 3s/step - loss: 6.8420 - acc: 0.0000e+00 - val_loss: 52.1337 - val_acc: 0.0000e+00
Epoch 6/15
4/4 [==============================] - 9s 2s/step - loss: 7.6581 - acc: 0.2500 - val_loss: 56.6134 - val_acc: 0.0000e+00
Epoch 7/15
4/4 [==============================] - 9s 2s/step - loss: 8.3480 - acc: 0.2500 - val_loss: 56.7590 - val_acc: 0.0000e+00
Epoch 8/15
4/4 [=

In [36]:
rgb_model_json = rgb_model.to_json()

In [37]:
with open("rgb_model.json", "w") as json_file:
    json_file.write(rgb_model_json)
# serialize weights to HDF5
rgb_model.save_weights("rgb_model.h5")
print("Saved model to disk")

Saved model to disk


In [38]:
# load json and create model
from keras.models import model_from_json
rgb_json_file = open('rgb_model.json', 'r')
rgb_loaded_model_json = rgb_json_file.read()
rgb_json_file.close()
rgb_loaded_model = model_from_json(rgb_loaded_model_json)
# load weights into new model
rgb_loaded_model.load_weights("rgb_model.h5")
print("Loaded model from disk")

Loaded model from disk


In [39]:
# evaluate loaded model on test data
rgb_loaded_model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
rgb_score = rgb_loaded_model.evaluate(X_train_rgb, y_train, verbose=0)
print("%s: %.2f%%" % (rgb_loaded_model.metrics_names[1], rgb_score[1]*100))

acc: 99.94%
